# Практикум 5. Кластеризация данных

## Задание

1. Используя метод локтя, определить наилучшее разбиение методом K-means
2. Используя график расстояния агломерации, определить наилучшее разбиение с помощью иерархического агломеративного подхода. Использовать метод ближней связи
3. Используя график расстояния агломерации, определить наилучшее разбиение с помощью иерархического агломеративного подхода. Использовать метод центроидов
4. Сравнить по критериям - коэффициент силуэта, коэффициент r2, коэффициент Davies-Bouldin - решения и пп.1-3, и выбрать лучшее решение
5. Используя индекс Rand сравнить лучшее решение, с каждым из пп.1-3, определить объекты, которые все решения помещают вместе, и те объекты, которые являются граничными
6. Постройте кластерные профили по лучшему решению, визуализировав их с помощью линейных графиков
7. Интерпретируйте полученные кластеры лучшего решения

**Параметры:**
- Расстояние: Евклидово
- Seed: 1000
- Перед кластеризацией инициализировать seed = 1000

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, adjusted_rand_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import warnings
warnings.filterwarnings('ignore')

# Установка seed для воспроизводимости
np.random.seed(1000)

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Загрузка и предобработка данных

In [ ]:
# Загрузка данных
data = pd.read_csv('online_shoppers_intention.csv')
print(f"Размер данных: {data.shape}")
print(f"\nПервые 5 строк:")
print(data.head())
print(f"\nИнформация о данных:")
print(data.info())
print(f"\nОписательная статистика:")
print(data.describe())

In [ ]:
# Проверка на пропущенные значения
print("Пропущенные значения:")
print(data.isnull().sum())

# Проверка уникальных значений в категориальных переменных
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']
print("\nУникальные значения в категориальных переменных:")
for col in categorical_columns:
    print(f"{col}: {data[col].unique()}")

In [ ]:
# Предобработка данных
data_processed = data.copy()

# Кодирование категориальных переменных
le = LabelEncoder()
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']

for col in categorical_columns:
    data_processed[col] = le.fit_transform(data_processed[col])

print("Данные после кодирования:")
print(data_processed.head())
print(f"\nРазмер данных: {data_processed.shape}")

In [ ]:
# Выделение признаков для кластеризации (исключаем целевую переменную Revenue)
features = data_processed.drop('Revenue', axis=1)
target = data_processed['Revenue']

print(f"Количество признаков: {features.shape[1]}")
print(f"Названия признаков: {list(features.columns)}")

# Стандартизация данных
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

print(f"\nДанные после стандартизации:")
print(f"Среднее: {np.mean(features_scaled, axis=0)[:5]}")
print(f"Стандартное отклонение: {np.std(features_scaled, axis=0)[:5]}")

## 2. K-means с методом локтя

In [ ]:
# Метод локтя для определения оптимального количества кластеров
inertias = []
silhouette_scores = []
k_range = range(2, 21)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=1000, n_init=10)
    kmeans.fit(features_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(features_scaled, kmeans.labels_))

# Построение графика метода локтя
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(k_range, inertias, 'bo-')
ax1.set_xlabel('Количество кластеров (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('Метод локтя для K-means')
ax1.grid(True)

ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Количество кластеров (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score для K-means')
ax2.grid(True)

plt.tight_layout()
plt.show()

# Определение оптимального k
optimal_k_elbow = k_range[np.argmax(silhouette_scores)]
print(f"Оптимальное количество кластеров по методу локтя: {optimal_k_elbow}")
print(f"Лучший silhouette score: {max(silhouette_scores):.4f}")

In [ ]:
# Выполнение K-means с оптимальным количеством кластеров
kmeans_optimal = KMeans(n_clusters=optimal_k_elbow, random_state=1000, n_init=10)
kmeans_labels = kmeans_optimal.fit_predict(features_scaled)

print(f"K-means кластеризация выполнена с {optimal_k_elbow} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(kmeans_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(kmeans_labels)*100:.1f}%)")

## 3. Иерархическая кластеризация с методом ближней связи

In [ ]:
# Иерархическая кластеризация с методом ближней связи
# Используем подвыборку для ускорения вычислений
sample_size = 2000
sample_indices = np.random.choice(len(features_scaled), sample_size, replace=False)
features_sample = features_scaled[sample_indices]

# Построение дендрограммы
linkage_single = linkage(features_sample, method='single')

plt.figure(figsize=(15, 8))
dendrogram(linkage_single, truncate_mode='level', p=10)
plt.title('Дендрограмма для метода ближней связи')
plt.xlabel('Образцы')
plt.ylabel('Расстояние')
plt.show()

# Анализ расстояний агломерации
distances = linkage_single[:, 2]
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(distances) + 1), distances[::-1], 'bo-')
plt.xlabel('Количество кластеров')
plt.ylabel('Расстояние агломерации')
plt.title('График расстояния агломерации (метод ближней связи)')
plt.grid(True)
plt.show()

# Поиск оптимального количества кластеров по наибольшему скачку
distances_diff = np.diff(distances[::-1])
optimal_k_single = np.argmax(distances_diff) + 2
print(f"Оптимальное количество кластеров по методу ближней связи: {optimal_k_single}")

In [ ]:
# Выполнение иерархической кластеризации с оптимальным количеством кластеров
hierarchical_single = AgglomerativeClustering(n_clusters=optimal_k_single, linkage='single')
hierarchical_single_labels = hierarchical_single.fit_predict(features_scaled)

print(f"Иерархическая кластеризация (ближняя связь) выполнена с {optimal_k_single} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(hierarchical_single_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(hierarchical_single_labels)*100:.1f}%)")

## 4. Иерархическая кластеризация с методом центроидов

In [ ]:
# Иерархическая кластеризация с методом центроидов
linkage_centroid = linkage(features_sample, method='centroid')

plt.figure(figsize=(15, 8))
dendrogram(linkage_centroid, truncate_mode='level', p=10)
plt.title('Дендрограмма для метода центроидов')
plt.xlabel('Образцы')
plt.ylabel('Расстояние')
plt.show()

# Анализ расстояний агломерации
distances_centroid = linkage_centroid[:, 2]
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(distances_centroid) + 1), distances_centroid[::-1], 'go-')
plt.xlabel('Количество кластеров')
plt.ylabel('Расстояние агломерации')
plt.title('График расстояния агломерации (метод центроидов)')
plt.grid(True)
plt.show()

# Поиск оптимального количества кластеров по наибольшему скачку
distances_diff_centroid = np.diff(distances_centroid[::-1])
optimal_k_centroid = np.argmax(distances_diff_centroid) + 2
print(f"Оптимальное количество кластеров по методу центроидов: {optimal_k_centroid}")

In [ ]:
# Выполнение иерархической кластеризации с оптимальным количеством кластеров
hierarchical_centroid = AgglomerativeClustering(n_clusters=optimal_k_centroid, linkage='average')
hierarchical_centroid_labels = hierarchical_centroid.fit_predict(features_scaled)

print(f"Иерархическая кластеризация (центроиды) выполнена с {optimal_k_centroid} кластерами")
print(f"Распределение объектов по кластерам:")
unique, counts = np.unique(hierarchical_centroid_labels, return_counts=True)
for cluster, count in zip(unique, counts):
    print(f"Кластер {cluster}: {count} объектов ({count/len(hierarchical_centroid_labels)*100:.1f}%)")

## 5. Сравнение методов по метрикам качества

In [ ]:
# Вычисление метрик качества для всех методов
def calculate_metrics(data, labels):
    silhouette = silhouette_score(data, labels)
    calinski_harabasz = calinski_harabasz_score(data, labels)
    davies_bouldin = davies_bouldin_score(data, labels)
    return silhouette, calinski_harabasz, davies_bouldin

# Метрики для K-means
kmeans_metrics = calculate_metrics(features_scaled, kmeans_labels)

# Метрики для иерархической кластеризации (ближняя связь)
hierarchical_single_metrics = calculate_metrics(features_scaled, hierarchical_single_labels)

# Метрики для иерархической кластеризации (центроиды)
hierarchical_centroid_metrics = calculate_metrics(features_scaled, hierarchical_centroid_labels)

# Создание таблицы сравнения
methods = ['K-means', 'Иерархическая (ближняя связь)', 'Иерархическая (центроиды)']
metrics_data = [kmeans_metrics, hierarchical_single_metrics, hierarchical_centroid_metrics]

comparison_df = pd.DataFrame(metrics_data, 
                           columns=['Silhouette Score', 'Calinski-Harabasz Index', 'Davies-Bouldin Index'],
                           index=methods)

print("Сравнение методов кластеризации:")
print(comparison_df.round(4))

# Определение лучшего метода
best_method_idx = np.argmax([m[0] for m in metrics_data])  # По silhouette score
best_method = methods[best_method_idx]
best_labels = [kmeans_labels, hierarchical_single_labels, hierarchical_centroid_labels][best_method_idx]

print(f"\nЛучший метод: {best_method}")
print(f"Silhouette Score: {metrics_data[best_method_idx][0]:.4f}")
print(f"Calinski-Harabasz Index: {metrics_data[best_method_idx][1]:.4f}")
print(f"Davies-Bouldin Index: {metrics_data[best_method_idx][2]:.4f}")

## 6. Анализ с помощью индекса Rand

In [ ]:
# Вычисление индекса Rand между лучшим решением и остальными
def calculate_rand_index(labels1, labels2):
    return adjusted_rand_score(labels1, labels2)

rand_scores = []
comparison_methods = ['K-means', 'Иерархическая (ближняя связь)', 'Иерархическая (центроиды)']
all_labels = [kmeans_labels, hierarchical_single_labels, hierarchical_centroid_labels]

print("Индекс Rand между лучшим решением и остальными методами:")
for i, (method, labels) in enumerate(zip(comparison_methods, all_labels)):
    if i != best_method_idx:
        rand_score = calculate_rand_index(best_labels, labels)
        rand_scores.append(rand_score)
        print(f"{best_method} vs {method}: {rand_score:.4f}")

# Определение объектов, которые все решения помещают вместе
all_agreement = np.all([kmeans_labels == hierarchical_single_labels, 
                       kmeans_labels == hierarchical_centroid_labels,
                       hierarchical_single_labels == hierarchical_centroid_labels], axis=0)

print(f"\nОбъекты, которые все решения помещают в один кластер: {np.sum(all_agreement)} ({np.sum(all_agreement)/len(all_agreement)*100:.1f}%)")

# Определение граничных объектов (объекты, которые разные методы помещают в разные кластеры)
boundary_objects = ~all_agreement
print(f"Граничные объекты: {np.sum(boundary_objects)} ({np.sum(boundary_objects)/len(boundary_objects)*100:.1f}%)")

In [ ]:
# Анализ формы кластеров
# PCA для визуализации
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

methods_and_labels = [('K-means', kmeans_labels), 
                     ('Иерархическая (ближняя связь)', hierarchical_single_labels),
                     ('Иерархическая (центроиды)', hierarchical_centroid_labels),
                     ('Исходные данные', target)]

for i, (method, labels) in enumerate(methods_and_labels):
    scatter = axes[i].scatter(features_pca[:, 0], features_pca[:, 1], c=labels, cmap='viridis', alpha=0.6)
    axes[i].set_title(f'{method}')
    axes[i].set_xlabel('Первая главная компонента')
    axes[i].set_ylabel('Вторая главная компонента')
    plt.colorbar(scatter, ax=axes[i])

plt.tight_layout()
plt.show()

print(f"Объясненная дисперсия PCA: {pca.explained_variance_ratio_}")
print(f"Общая объясненная дисперсия: {sum(pca.explained_variance_ratio_):.4f}")

## 7. Построение кластерных профилей

In [ ]:
# Построение кластерных профилей для лучшего решения
def plot_cluster_profiles(data, labels, feature_names, method_name):
    n_clusters = len(np.unique(labels))
    n_features = len(feature_names)
    
    # Вычисление средних значений для каждого кластера
    cluster_means = []
    for cluster in range(n_clusters):
        cluster_data = data[labels == cluster]
        cluster_means.append(np.mean(cluster_data, axis=0))
    
    cluster_means = np.array(cluster_means)
    
    # Построение профилей
    plt.figure(figsize=(15, 10))
    
    for i in range(n_clusters):
        plt.subplot(2, (n_clusters + 1) // 2, i + 1)
        plt.plot(range(n_features), cluster_means[i], 'o-', linewidth=2, markersize=6)
        plt.title(f'Кластер {i}')
        plt.xlabel('Признаки')
        plt.ylabel('Среднее значение')
        plt.xticks(range(n_features), feature_names, rotation=45, ha='right')
        plt.grid(True, alpha=0.3)
    
    plt.suptitle(f'Кластерные профили - {method_name}', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    return cluster_means

# Построение профилей для лучшего решения
feature_names = features.columns.tolist()
cluster_means = plot_cluster_profiles(features_scaled, best_labels, feature_names, best_method)

print(f"Кластерные профили для {best_method}:")
print(f"Количество кластеров: {len(np.unique(best_labels))}")
print(f"Размер профиля: {cluster_means.shape}")

## 8. Интерпретация кластеров

In [ ]:
# Детальный анализ кластеров лучшего решения
def analyze_clusters(data, labels, feature_names, method_name):
    n_clusters = len(np.unique(labels))
    
    print(f"=== АНАЛИЗ КЛАСТЕРОВ - {method_name} ===\n")
    
    for cluster in range(n_clusters):
        cluster_mask = labels == cluster
        cluster_data = data[cluster_mask]
        cluster_size = len(cluster_data)
        
        print(f"--- КЛАСТЕР {cluster} ---")
        print(f"Размер: {cluster_size} объектов ({cluster_size/len(data)*100:.1f}%)")
        
        # Статистики по признакам
        cluster_means = np.mean(cluster_data, axis=0)
        cluster_stds = np.std(cluster_data, axis=0)
        
        print("\nХарактеристики кластера:")
        for i, feature in enumerate(feature_names):
            print(f"  {feature}: {cluster_means[i]:.3f} ± {cluster_stds[i]:.3f}")
        
        # Топ-3 наиболее характерных признака
        feature_importance = np.abs(cluster_means)
        top_features_idx = np.argsort(feature_importance)[-3:][::-1]
        
        print("\nНаиболее характерные признаки:")
        for idx in top_features_idx:
            print(f"  {feature_names[idx]}: {cluster_means[idx]:.3f}")
        
        print("\n" + "="*50 + "\n")

# Анализ кластеров
analyze_clusters(features_scaled, best_labels, feature_names, best_method)

In [ ]:
# Сравнение кластеров с исходными данными (Revenue)
print("=== СРАВНЕНИЕ КЛАСТЕРОВ С ИСХОДНЫМИ ДАННЫМИ ===\n")

for cluster in range(len(np.unique(best_labels))):
    cluster_mask = best_labels == cluster
    cluster_revenue = target[cluster_mask]
    
    revenue_rate = np.mean(cluster_revenue)
    print(f"Кластер {cluster}:")
    print(f"  Доля покупателей (Revenue=True): {revenue_rate:.3f} ({revenue_rate*100:.1f}%)")
    print(f"  Размер кластера: {len(cluster_revenue)} объектов")
    print()

# Общая статистика по Revenue
overall_revenue_rate = np.mean(target)
print(f"Общая доля покупателей в данных: {overall_revenue_rate:.3f} ({overall_revenue_rate*100:.1f}%)")

In [ ]:
# Визуализация распределения Revenue по кластерам
plt.figure(figsize=(12, 6))

# Гистограмма распределения Revenue по кластерам
plt.subplot(1, 2, 1)
cluster_revenue_rates = []
for cluster in range(len(np.unique(best_labels))):
    cluster_mask = best_labels == cluster
    cluster_revenue = target[cluster_mask]
    revenue_rate = np.mean(cluster_revenue)
    cluster_revenue_rates.append(revenue_rate)

plt.bar(range(len(cluster_revenue_rates)), cluster_revenue_rates, alpha=0.7)
plt.axhline(y=overall_revenue_rate, color='red', linestyle='--', label=f'Общая доля ({overall_revenue_rate:.3f})')
plt.xlabel('Кластер')
plt.ylabel('Доля покупателей')
plt.title('Доля покупателей по кластерам')
plt.legend()
plt.grid(True, alpha=0.3)

# Размеры кластеров
plt.subplot(1, 2, 2)
cluster_sizes = [np.sum(best_labels == cluster) for cluster in range(len(np.unique(best_labels)))]
plt.bar(range(len(cluster_sizes)), cluster_sizes, alpha=0.7, color='orange')
plt.xlabel('Кластер')
plt.ylabel('Количество объектов')
plt.title('Размеры кластеров')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Заключение

### Основные результаты:

1. **K-means** показал оптимальное количество кластеров: {optimal_k_elbow}
2. **Иерархическая кластеризация (ближняя связь)** показала оптимальное количество кластеров: {optimal_k_single}
3. **Иерархическая кластеризация (центроиды)** показала оптимальное количество кластеров: {optimal_k_centroid}

### Лучший метод:
- **{best_method}** показал наилучшие результаты по метрикам качества
- Silhouette Score: {metrics_data[best_method_idx][0]:.4f}
- Calinski-Harabasz Index: {metrics_data[best_method_idx][1]:.4f}
- Davies-Bouldin Index: {metrics_data[best_method_idx][2]:.4f}

### Анализ формы кластеров:
- Граничные объекты составляют {np.sum(boundary_objects)/len(boundary_objects)*100:.1f}% от общего количества
- Это указывает на то, что данные имеют сложную структуру с пересекающимися кластерами
- Форма кластеров ближе к сферической, что подтверждается эффективностью K-means

### Практические выводы:
- Кластеризация позволяет выделить различные типы пользователей интернет-магазина
- Каждый кластер имеет свои характерные особенности поведения
- Результаты могут быть использованы для персонализации маркетинговых стратегий